In [5]:
#!pip install yfinance
#!pip install fbprophet
import datetime
from pathlib import Path

import joblib
import pandas as pd
import yfinance as yf
from fbprophet import Prophet
from fastapi import FastAPI

     |████████████████████████████████| 54 kB 260 kB/s eta 0:00:01
     |████████████████████████████████| 39.1 MB 6.2 kB/s eta 0:00:01
     |████████████████████████████████| 43 kB 358 kB/s eta 0:00:01
     |████████████████████████████████| 114 kB 209 kB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 127 kB/s eta 0:00:01
     |████████████████████████████████| 732 kB 181 kB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /Users/kateengard/opt/anaconda3/envs/PythonData/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/c4/xdvw79zn42b7mvxxm70kylt40000gn/T/pip-install-awc68ts8/fbprophet/setup.py'"'"'; __file__='"'"'/private/var/folders/c4/xdvw79zn42b7mvxxm70kylt40000gn/T/pip-install-awc68ts8/fbprophet/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folde

ModuleNotFoundError: No module named 'fbprophet'

In [ ]:
BASE_DIR = Path(__file__).resolve(strict=True).parent
TODAY = datetime.date.today()


def train(ticker="MSFT"):
    # data = yf.download("^GSPC", "2008-01-01", TODAY.strftime("%Y-%m-%d"))
    data = yf.download(ticker, "2020-01-01", TODAY.strftime("%Y-%m-%d"))
    data.head()
    data["Adj Close"].plot(title=f"{ticker} Stock Adjusted Closing Price")

    df_forecast = data.copy()
    df_forecast.reset_index(inplace=True)
    df_forecast["ds"] = df_forecast["Date"]
    df_forecast["y"] = df_forecast["Adj Close"]
    df_forecast = df_forecast[["ds", "y"]]
    df_forecast

    model = Prophet()
    model.fit(df_forecast)

    joblib.dump(model, Path(BASE_DIR).joinpath(f"{ticker}.joblib"))


def predict(ticker="MSFT", days=7):
    model_file = Path(BASE_DIR).joinpath(f"{ticker}.joblib")
    if not model_file.exists():
        return False

    model = joblib.load(model_file)

    future = TODAY + datetime.timedelta(days=days)

    dates = pd.date_range(start="2020-01-01", end=future.strftime("%m/%d/%Y"),)
    df = pd.DataFrame({"ds": dates})

    forecast = model.predict(df)

    model.plot(forecast).savefig(f"{ticker}_plot.png")
    model.plot_components(forecast).savefig(f"{ticker}_plot_components.png")

    return forecast.tail(days).to_dict("records")


def convert(prediction_list):
    output = {}
    for data in prediction_list:
        date = data["ds"].strftime("%m/%d/%Y")
        output[date] = data["trend"]
    return output